In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import Annotated

llm = init_chat_model("gpt-4o-mini")

In [3]:
# llm.invoke("What is the capital of France?")

In [4]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    
def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

In [5]:
builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [6]:
message = {    "role": "user",
    "content": "What is the capital of France?"
}
response = graph.invoke({"messages": [message]})

response["messages"]

[HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}, id='f689403a-66c3-4b0b-ab1b-fb1f18c0c056'),
 AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BnzkCOchdKKc1LJXtCMYH1Du7DvLT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d7af3846-f3ef-4738-8771-d41a8075ddd3-0', usage_metadata={'input_tokens': 14, 'output_tokens': 7, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [8]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"exit", "quit"}:
        print("Exiting the chatbot.")
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}] 
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Human:", state["messages"][-2].content)
    print("Bot:", state["messages"][-1].content)

You:  Who was the first person on moon? Give the name only.


Human: Who was the first person on moon? Give the name only.
Bot: Neil Armstrong.


You:  which year


Human: which year
Bot: 1969.


You:  quit


Exiting the chatbot.


In [7]:
message = {    "role": "user",
    "content": "What is the latest price of MSFT stock?"
}
response = graph.invoke({"messages": [message]})

response["messages"]

[HumanMessage(content='What is the latest price of MSFT stock?', additional_kwargs={}, response_metadata={}, id='49f3773e-5faa-4fb0-96db-88380f7f5222'),
 AIMessage(content="I don't have access to real-time data, including current stock prices. To find the latest price of Microsoft Corporation (MSFT) stock, I recommend checking a financial news website, stock market app, or your brokerage platform.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 17, 'total_tokens': 62, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bnzl7wXTmNRv9ZfkYKI7c1DKCtqok', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2ed7b10a-1bc8-4c02-8ef5-e670d47ee0bd-0', usage